## Inflation Price Analytics

- query the API of the US Bureau of Labor Statistics
- format time series data
- plot and compare time series data

In [ ]:
_ = !pip install matplotlib
_ = !pip install pandas
_ = !pip install requests

In [ ]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# get API key at https://www.bls.gov/
from apikey import bls

In [ ]:
# information on data available
# https://www.bls.gov/help/hlpforma.htm#AP

In [ ]:
series_map = {
    'steak [lb]': 'APU0000703511',
    'eggs [dozen]': 'APU0000708111',
    'tomatoes [lb]': 'APU0000712311',
    'dried beans [lb]': 'APU0000714233',
    'electricity [kWh]': 'APU000072610',
    'gasoline, unleaded regular [gallon]': 'APU000074714'
}
for k, v in list(series_map.items()):
    series_map[v] = k
series_map

In [ ]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
headers = {'Content-type': 'application/json'}
data = json.dumps(
    {'seriesid': [
        series_map['steak [lb]'],
        series_map['eggs [dozen]'],
        series_map['tomatoes [lb]'],
        series_map['dried beans [lb]'],
        series_map['electricity [kWh]'],
        series_map['gasoline, unleaded regular [gallon]'],
    ], 
     'registrationkey': bls, 
     'startyear':'2019', 
     'endyear':'2023'})

# Send API request
response = requests.post(url, data=data, headers=headers)

In [ ]:
response

In [ ]:
response.json()

In [ ]:
series_list = response.json()['Results']['series']

In [ ]:
data = {
    'item': [],
    'year-month': [],
    'price': []
}

In [ ]:
for series in series_list:
    item = series_map[series['seriesID']]
    for record in series['data']:
        if not record['period'].startswith('M'):
            continue
        month = record['period'][1:]
        year = record['year']
        year_month = f'{year}-{month}'
        price = float(record['value'])
        data['item'].append(item)
        data['year-month'].append(year_month)
        data['price'].append(price)        

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
df['year-month'] = pd.to_datetime(df['year-month'], format='%Y-%m')
df.set_index('year-month', inplace=True)

In [ ]:
grouped = df.groupby('item')

for name, group in grouped:
    group['price'].plot(label=name)

plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Price over Time')
plt.xticks(rotation = 90)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
grouped = df.groupby('item')

for name, group in grouped:
    starting_price = group.iloc[-1]['price']
    group['price_normalized'] = group['price'] / starting_price
    group['price_normalized'].plot(label=name)

plt.xlabel('Time')
plt.ylabel('Normalized Price')
plt.title('Normalized Price over Time')
plt.xticks(rotation = 90)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()